In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import easydel as ed
import jax

HELLO FROM EASYDEL


2025-12-09 15:41:02,873 - numexpr.utils - INFO - Note: detected 256 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2025-12-09 15:41:02,874 - numexpr.utils - INFO - Note: NumExpr detected 256 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2025-12-09 15:41:02,874 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.
2025-12-09 15:41:03,737 - eray-executor - INFO - Not initializing jax.distributed because no distributed config was provided, and no cluster was detected.
2025-12-09 15:41:03,739 - eray-executor - INFO - No auto-discovered ray address found. Using ray.init('local').
2025-12-09 15:41:03,739 - eray-executor - INFO - ray.init(address='local', namespace='eray-executor', **{})
2025-12-09 15:41:06,912	INFO worker.py:1908 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [2]:
checkpoint_dir = "/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-3b/gidd/orbax"
# checkpoint_dir = "/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-10b/gidd/orbax"
# checkpoint_dir = "/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-mask-3b/gidd/orbax"

In [3]:
state_dict_path = os.path.join(checkpoint_dir, "..", "state_dict.pt")

if "gidd-unif-3b" in checkpoint_dir:
    num_layers = 19
    hidden_size = 3072
    num_attn_heads = 24
elif "gidd-mask-3b" in checkpoint_dir:
    num_layers = 18
    hidden_size = 3072
    num_attn_heads = 24
elif "gidd-unif-10b" in checkpoint_dir:
    num_layers = 34
    hidden_size = 4608
    num_attn_heads = 36
else:
    raise ValueError("Unknown checkpoint")

In [4]:
from gidd_easydel.loading import load_checkpoint

mesh, module, tokenizer, dtype = load_checkpoint(
    checkpoint_dir,
    num_layers=num_layers,
    hidden_size=hidden_size,
    num_attn_heads=num_attn_heads,
    precision=jax.lax.Precision.HIGHEST,
)

HELLO FROM ORBAX


ERROR:2025-12-09 15:41:15,728:jax._src.xla_bridge:487: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/local/home/dvruette/gidd-easydel/.venv/lib/python3.11/site-packages/jax/_src/xla_bridge.py", line 485, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/local/home/dvruette/gidd-easydel/.venv/lib/python3.11/site-packages/jax_plugins/xla_cuda12/__init__.py", line 328, in initialize
    _check_cuda_versions(raise_on_first_error=True)
  File "/local/home/dvruette/gidd-easydel/.venv/lib/python3.11/site-packages/jax_plugins/xla_cuda12/__init__.py", line 285, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: CUDA_ERROR_NO_DEVICE
2025-12-09 15:41:15,728 - jax._src.xla_bridge - ERROR - Jax plugin configuration error: Excepti

/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-3b/gidd/orbax


2025-12-09 15:41:32,377 - absl - INFO - [process=0] /jax/checkpoint/read/gbytes_per_sec: 3 Bytes/s (total gbytes: 5.5 GiB) (time elapsed: a second) (per-host)
2025-12-09 15:41:32,380 - absl - INFO - Finished restoring checkpoint in 1.57 seconds from /pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-3b/gidd/orbax/22500.
2025-12-09 15:41:32,381 - absl - INFO - {'step': 22500, 'event_type': 'restore', 'directory': '/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-3b/gidd/orbax', 'checkpointer_start_time': 1765291290.8074493, 'checkpointer_duration_secs': 1.5735054016113281, 'checkpoint_manager_start_time': 1765291290.802595, 'checkpoint_manager_duration_secs': 1.578361988067627}
2025-12-09 15:41:32,381 - absl - INFO - [process=0][thread=MainThread][wait_until_finished] No Save Finalize thread to wait for. Returning.
2025-12-09 15:41:32,382 - absl - INFO - Closing _NonBlockingMetadataStore(enable_write=True, _write_lock=<locked _thread.RLock object owner=140311120570176 count=1 at 0

In [5]:
import jax.numpy as jnp

input_ids = jnp.arange(8)[None, :]
noise_mask = jnp.ones((1, 8), dtype=jnp.bool)
noise_mask = noise_mask.at[:, :noise_mask.shape[1] // 2].set(False)
attn_mask = (noise_mask[..., None] >= noise_mask[..., None, :])

with mesh:
    outputs = module(
        input_ids=input_ids,
        attention_mask=attn_mask,
        output_hidden_states=True,
    )

jnp.save("hidden_states.npy", outputs.hidden_states)
jnp.save("logits.npy", outputs.logits)

hidden_states [[[-0.00897217 5.74589e-05 -0.0275879 ... -0.00408936 -0.00976562
   0.0473633]
  [-0.0281982 0.0478516 -0.0495605 ... -0.0476074 -0.0251465 -0.0732422]
  [0.0145874 0.0844727 0.0368652 ... 0.00689697 0.00515747 0.0510254]
  ...
  [-0.00148773 0.00230408 -0.0361328 ... 0.0388184 0.00402832 0.0146484]
  [-0.0388184 0.0617676 0.0280762 ... -0.0201416 -0.0241699 0.0717773]
  [0.00656128 -0.0177002 -0.0615234 ... -0.00939941 -0.0150146 0.0703125]]]
Using separate lm_head for lm_head
logits [[[-8.8125 1.17969 -8.5625 ... -6.1875 -4.03125 -3.28125]
  [-8.4375 3.84375 -9.125 ... -6.96875 -4.125 -5.8125]
  [-8 5.6875 -8.25 ... -4.8125 -5.8125 -1.26562]
  ...
  [-6.9375 7.6875 -7.84375 ... -5.96875 -9.875 -4.1875]
  [-4.96875 9.1875 -6.46875 ... -4.375 -6.625 -4.6875]
  [-5.6875 7.4375 -6.5 ... -4.4375 -7.9375 -2.71875]]]


In [6]:
outputs.logits[0, [0, 1, 2, 3, 4, 5, 6, 7], [0, 1, 2, 3, 4, 5, 6, 7]]

Array([-8.8125, 3.84375, -8.25, 0, 11.875, 10.3125, 8, 3.5], dtype=bfloat16)

In [7]:
outputs.hidden_states[1][0, 1, 222]

Array(-0.03125, dtype=bfloat16)

In [8]:
outputs.hidden_states[-1]

Array([[[-0.00897217, 5.74589e-05, -0.0275879, ..., -0.00408936,
         -0.00976562, 0.0473633],
        [-0.0281982, 0.0478516, -0.0495605, ..., -0.0476074, -0.0251465,
         -0.0732422],
        [0.0145874, 0.0844727, 0.0368652, ..., 0.00689697, 0.00515747,
         0.0510254],
        ...,
        [-0.00148773, 0.00230408, -0.0361328, ..., 0.0388184,
         0.00402832, 0.0146484],
        [-0.0388184, 0.0617676, 0.0280762, ..., -0.0201416, -0.0241699,
         0.0717773],
        [0.00656128, -0.0177002, -0.0615234, ..., -0.00939941,
         -0.0150146, 0.0703125]]], dtype=bfloat16)

In [9]:
outputs.logits

Array([[[-8.8125, 1.17969, -8.5625, ..., -6.1875, -4.03125, -3.28125],
        [-8.4375, 3.84375, -9.125, ..., -6.96875, -4.125, -5.8125],
        [-8, 5.6875, -8.25, ..., -4.8125, -5.8125, -1.26562],
        ...,
        [-6.9375, 7.6875, -7.84375, ..., -5.96875, -9.875, -4.1875],
        [-4.96875, 9.1875, -6.46875, ..., -4.375, -6.625, -4.6875],
        [-5.6875, 7.4375, -6.5, ..., -4.4375, -7.9375, -2.71875]]],      dtype=bfloat16)